In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
with open('geodata/subdivisions.html', 'r') as f:
    subdivisions_html = f.read()

In [3]:
df = pd.read_csv('data/all_buurt.csv')

In [4]:
len(df)

513

In [5]:
soup = BeautifulSoup(subdivisions_html, "html.parser")

neighborhood_to_subdivision = {}
neighborhood_to_stadsdeel = {}

current_subdivision = None

for tag in soup.find_all(["h3", "b", "li"]):
    if tag.name == "h3":
        # Use <h3> tag text as the current subdivision
        current_subdivision = tag.text.strip()
        if 'Stadsdeel' in current_subdivision or current_subdivision in ['Westpoort', 'Weesp']:
            current_stadsteel = current_subdivision
            print('**STADSDEEL', current_stadsteel, '**')
            continue
        print(current_subdivision)
    elif tag.name == "b":
        # Use <b> tag text as the current subdivision
        current_subdivision = tag.text.strip()
        if 'Stadsdeel' in current_subdivision or current_subdivision in ['Westpoort', 'Weesp']:
            current_stadsteel = current_subdivision
            print('**STADSDEEL', current_stadsteel, '**')
            continue
        print(current_subdivision)
    elif tag.name == "li" and current_subdivision:
        # Extract neighborhoods under the current subdivision
        neighborhood = tag.text.strip()
        if '\n' in neighborhood:
            continue
        neighborhood_to_subdivision[neighborhood] = current_subdivision
        neighborhood_to_stadsdeel[neighborhood] = current_stadsteel

**STADSDEEL Stadsdeel Centrum **
Centrum-West
Centrum-Oost
**STADSDEEL Stadsdeel Noord **
Oost
Oud-Noord
West
**STADSDEEL Stadsdeel West **
Bos en Lommer
Oud West / De Baarsjes
Westerpark
**STADSDEEL Stadsdeel Nieuw-West **
De Aker, Sloten en Nieuw Sloten
Geuzenveld-Slotermeer-Sloterdijken
Osdorp
Slotervaart
**STADSDEEL Stadsdeel Zuid **
Buitenveldert / Zuidas
De Pijp / Rivierenbuurt
Oud Zuid
**STADSDEEL Stadsdeel Oost **
IJburg / Eiland Zeeburg
Indische Buurt / Oostelijk Havengebied
Oud-Oost
Watergraafsmeer
**STADSDEEL Stadsdeel Zuidoost **
Bijlmer Centrum
Bijlmer Oost
Gaasperdam
**STADSDEEL Westpoort **
**STADSDEEL Weesp **


In [6]:
buurt_mapping_wp = pd.DataFrame.from_dict(
    neighborhood_to_subdivision, 
    orient='index', columns=['stadsdeel_onderverdeling']).join(
    pd.DataFrame.from_dict(
        neighborhood_to_stadsdeel, 
        orient='index', columns=['stadsdeel'])
)
buurt_mapping_wp = buurt_mapping_wp.reset_index().rename(columns={'index': 'buurt'})

In [7]:
buurt_mapping_wp

,buurt,stadsdeel_onderverdeling,stadsdeel
0,BG-terrein e.o.,Centrum-West,Stadsdeel Centrum
1,Burgwallen oost,Centrum-West,Stadsdeel Centrum
2,Kop Zeedijk,Centrum-West,Stadsdeel Centrum
3,Nes e.o.,Centrum-West,Stadsdeel Centrum
4,Oude Kerk e.o.,Centrum-West,Stadsdeel Centrum
...,...,...,...
473,Westhaven Zuid,Westpoort,Westpoort
474,Horn (Amsterdam),Weesp,Weesp
475,Uitermeer,Weesp,Weesp
476,Dorp Driemond,Weesp,Weesp


In [10]:
df_join_1 = df.set_index('buurt').join(buurt_mapping_wp.set_index('buurt')).reset_index()

In [11]:
unmatched = df_join_1[df_join_1['stadsdeel'].isna()][['buurt', 'wijk']]

In [12]:
attempt_buurtname_correction = pd.DataFrame(unmatched)
attempt_buurtname_correction['buurt_correction'] = attempt_buurtname_correction['buurt'].str.replace('-', ' ', regex=False).str.replace('/', ' / ', regex=False)

In [14]:
df_join_2 = attempt_buurtname_correction.set_index('buurt_correction').join(buurt_mapping_wp.set_index('buurt')).reset_index(drop=True)

In [15]:
still_unmatched = df_join_2[df_join_2['stadsdeel'].isna()]['buurt']

In [16]:
buurt_unmatched_to_subdivision = {
    'Reigersbos 1' : 'Gaasperdam', 
    'Noordoever Sloterplas': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Reigersbos 2' : 'Gaasperdam', 
    'Gein 1': 'Gaasperdam',
    'Reigersbos 3' : 'Gaasperdam', 
    'Dijkgraafpleinbuurt' : 'Osdorp', 
    'GWL-terrein': 'Westerpark',
    'G-buurt-West': 'Bijlmer Oost', 
    'Louis Couperusbuurt': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'De Kameleon e.o.': 'Bijlmer Oost',
    'Holendrecht-Zuid': 'Gaasperdam' , 
    'Gein 4': 'Gaasperdam', 
    'Dichtersbuurt': 'Geuzenveld-Slotermeer-Sloterdijken',
    'Louis Crispijnbuurt': 'Slotervaart', 
    'Reigersbos 4': 'Gaasperdam', 
    'Gein 2': 'Gaasperdam',
    'K-buurt-Zuidwest': 'Bijlmer Oost', 
    'Gein 3': 'Gaasperdam', 
    'Hemelrijk': 'Centrum-West', 
    'Burgwallen-Oost': 'Centrum-West',
    'Vijzelstraatbuurt': 'Centrum-Oost', 
    'Rembrandtplein e.o.': 'Centrum-Oost', 
    'Geelvinckbuurt': 'Centrum-Oost',
    'Minervahaven-Zuidwest': 'Westelijk Havengebied', 
    'Minervahaven-Zuidoost': 'Westelijk Havengebied',
    'Sloterdijk Stationskwartier': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Van Tijenbuurt': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Dudokbuurt': 'Geuzenveld-Slotermeer-Sloterdijken',
    'Ruys de Beerenbrouckbuurt': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Wegener Sleeswijkbuurt': 'Geuzenveld-Slotermeer-Sloterdijken',
    'Bakemabuurt': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'De Eendracht': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Osdorper Binnenpolder': 'Geuzenveld-Slotermeer-Sloterdijken',
    'Jan de Louterbuurt': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Lodewijk van Deysselbuurt': 'Geuzenveld-Slotermeer-Sloterdijken',
    'Confuciusbuurt': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Dobbebuurt': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Coronelbuurt': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Arondeusbuurt': 'Geuzenveld-Slotermeer-Sloterdijken',
    'Zuidwestkwadrant-Zuid': 'Osdorp', 
    'Zuidwestkwadrant-Noord': 'Osdorp',
    'Botteskerkbuurt': 'Osdorp', 
    'Reimerswaal': 'Osdorp', 
    'Osdorpplein e.o.': 'Osdorp',
    'Calandlaan/Lelylaan': 'Osdorp', 
    'Lucas/Andreasziekenhuis e.o.': 'Slotervaart',
    'Dorp Sloten': 'De Aker, Sloten en Nieuw Sloten', 
    'Belgiëplein e.o.': 'De Aker, Sloten en Nieuw Sloten',
    'BurgemeesterTellegenbuurt-West': 'De Pijp / Rivierenbuurt',
    'Veluwebuurt': 'De Pijp / Rivierenbuurt',
    'Buitenveldert-Zuid-Midden': 'Buitenveldert / Zuidas', 
    'Cruquiusbuurt': 'Indische Buurt / Oostelijk Havengebied',
    'Weesperzijde-Midden/Zuid' : 'Oud-Oost', 
    'Weesperzijde-Noord': 'Oud-Oost',
    'Timorpleinbuurt-Noord': 'Indische Buurt / Oostelijk Havengebied', 
    'Ambonpleinbuurt': 'Indische Buurt / Oostelijk Havengebied', 
    'Timorpleinbuurt-Zuid': 'Indische Buurt / Oostelijk Havengebied',
    'Makassarpleinbuurt': 'Indische Buurt / Oostelijk Havengebied', 
    'Sumatraplantsoenbuurt': 'Indische Buurt / Oostelijk Havengebied', 
    'Sportheldenbuurt': 'IJburg / Eiland Zeeburg',
    'Oostpunt Zeeburgereiland': 'IJburg / Eiland Zeeburg', 
    'Sluisbuurt': 'IJburg / Eiland Zeeburg', 
    'Baaibuurt-West': 'IJburg / Eiland Zeeburg',
    'Bovendiep/Diemerpark': 'IJburg / Eiland Zeeburg', 
    'Joris Ivenspleinbuurt': 'IJburg / Eiland Zeeburg',
    'Ed Pelsterparkbuurt': 'IJburg / Eiland Zeeburg', 
    'Rieteilanden-West': 'IJburg / Eiland Zeeburg', 
    'Havenkwartier IJburg': 'IJburg / Eiland Zeeburg',
    'Theo van Goghparkbuurt': 'IJburg / Eiland Zeeburg', 
    'Circus/Kermisbuurt': 'West',
    'Buiksloterham-Noord': 'Oud-Noord', 
    'Buiksloterham-Zuid': 'Oud-Noord', 
    'Klaprozenbuurt': 'Oud-Noord',
    'Markengouw-Noord': 'Oost', 
    'G-buurt-Oost': 'Bijlmer Oost', 
    'Grunder/Koningshoef' : 'Bijlmer Oost',
    'Amstel III deel A/B-Noord': 'Bijlmer Centrum',
    'Amstel III deel A/B-Zuid': 'Bijlmer Centrum',
    'Afrikahaven' : 'Westpoort', 
    'Coenhaven' : 'Westpoort', 
    'Vlothaven': 'Westpoort', 
    'Minervahaven-Noordoost': 'Westpoort',
    'Minervahaven-Noordwest': 'Westpoort', 
    'Hemknoop-Noord': 'Westpoort', 
    'Isolatorbuurt': 'Bos en Lommer',
    'Contactbuurt': 'Bos en Lommer', 
    'Hemknoop-Zuid': 'Westerpark', 
    'Zaanstraat emplacement': 'Westerpark',
    'Sloterdijk De Heining': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Sloterdijk Poort-Noord': 'Geuzenveld-Slotermeer-Sloterdijken',
    'Sloterdijk Poort-Zuid': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Sloterdijk Rhônepark': 'Geuzenveld-Slotermeer-Sloterdijken',
    'Sloterdijk Donauhoek': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Meerwaldtbuurt': 'Geuzenveld-Slotermeer-Sloterdijken', 
    'Baaibuurt-Oost': 'IJburg / Eiland Zeeburg',
    'Melkweg/Oostzanerwerf': 'West', 
    'Cornelis Douwesterrein-West': 'Oud-Noord',
    'Cornelis Douwesterrein-Oost': 'Oud-Noord', 
    'Werengouw-Noord': 'Oost',
    'Hamerstraatkwartier-West': 'Oud-Noord', 
    'Hamerstraatkwartier-Oost': 'Oud-Noord',
    'Weespersluis-Noord': 'Weesp', 
    'Waterrijk': 'Weesp', 
    'Lanenrijk': 'Weesp', 
    'Vechtrijk': 'Weesp',
    'Leeuwenveld': 'Weesp', 
    'Bedrijventerrein Nijverheidslaan': 'Weesp',
    'Bedrijventerrein-Noord Weesp': 'Weesp', 
    'Hogewey-Noord': 'Weesp', 
    'Hogewey-Midden': 'Weesp',
    'Hogewey-Zuid': 'Weesp', 
    'Weesp Dichtersbuurt': 'Weesp', 
    'Weesp Schildersbuurt': 'Weesp',
    'Herensingelkwartier': 'Weesp', 
    'Bedrijventerrein Van Houten': 'Weesp',
    'Weesp Centrum': 'Weesp', 
    'Weesp-Zuid I': 'Weesp', 
    'Weesp-Zuid II': 'Weesp', 
    'De Klomp': 'Weesp',
    'Aetsveldsepolder': 'Weesp', 
    'Aetsveld-Zuid': 'Weesp', 
    'Aetsveld-Noord': 'Weesp',
    'Oostelijke Vechtoever': 'Weesp', 
    'Amstel III deel C/D-Noord': 'Bijlmer Centrum',
    'Amstel III deel C/D-Zuid': 'Bijlmer Centrum', 
    'G-buurt-Noord': 'Bijlmer Oost', 
    'K-buurt-Zuidoost' : 'Bijlmer Oost'
}

In [17]:
buurt_unmatched_to_stadsdeel = {
    'Reigersbos 1' : 'Stadsdeel Zuidoost', 
    'Noordoever Sloterplas': 'Stadsdeel Nieuw-West', 
    'Reigersbos 2' : 'Stadsdeel Zuidoost', 
    'Gein 1': 'Stadsdeel Zuidoost',
    'Reigersbos 3' : 'Stadsdeel Zuidoost', 
    'Dijkgraafpleinbuurt' : 'Stadsdeel Nieuw-West', 
    'GWL-terrein': 'Stadsdeel West',
    'G-buurt-West': 'Stadsdeel Zuidoost', 
    'Louis Couperusbuurt': 'Stadsdeel Nieuw-West', 
    'De Kameleon e.o.': 'Stadsdeel Zuidoost',
    'Holendrecht-Zuid': 'Stadsdeel Zuidoost' , 
    'Gein 4': 'Stadsdeel Zuidoost', 
    'Dichtersbuurt': 'Stadsdeel Nieuw-West',
    'Louis Crispijnbuurt': 'Stadsdeel Nieuw-West', 
    'Reigersbos 4': 'Stadsdeel Zuidoost', 
    'Gein 2': 'Stadsdeel Zuidoost',
    'K-buurt-Zuidwest': 'Stadsdeel Zuidoost', 
    'Gein 3': 'Stadsdeel Zuidoost', 
    'Hemelrijk': 'Stadsdeel Centrum', 
    'Burgwallen-Oost': 'Stadsdeel Centrum',
    'Vijzelstraatbuurt': 'Stadsdeel Centrum', 
    'Rembrandtplein e.o.': 'Stadsdeel Centrum', 
    'Geelvinckbuurt': 'Stadsdeel Centrum',
    'Minervahaven-Zuidwest': 'Westpoort', 
    'Minervahaven-Zuidoost': 'Westpoort',
    'Sloterdijk Stationskwartier': 'Stadsdeel Nieuw-West', 
    'Van Tijenbuurt': 'Stadsdeel Nieuw-West', 
    'Dudokbuurt': 'Stadsdeel Nieuw-West',
    'Ruys de Beerenbrouckbuurt': 'Stadsdeel Nieuw-West', 
    'Wegener Sleeswijkbuurt': 'Stadsdeel Nieuw-West',
    'Bakemabuurt': 'Stadsdeel Nieuw-West', 
    'De Eendracht': 'Stadsdeel Nieuw-West', 
    'Osdorper Binnenpolder': 'Stadsdeel Nieuw-West',
    'Jan de Louterbuurt': 'Stadsdeel Nieuw-West', 
    'Lodewijk van Deysselbuurt': 'Stadsdeel Nieuw-West',
    'Confuciusbuurt': 'Stadsdeel Nieuw-West', 
    'Dobbebuurt': 'Stadsdeel Nieuw-West', 
    'Coronelbuurt': 'Stadsdeel Nieuw-West', 
    'Arondeusbuurt': 'Stadsdeel Nieuw-West',
    'Zuidwestkwadrant-Zuid': 'Stadsdeel Nieuw-West', 
    'Zuidwestkwadrant-Noord': 'Stadsdeel Nieuw-West',
    'Botteskerkbuurt': 'Stadsdeel Nieuw-West', 
    'Reimerswaal': 'Stadsdeel Nieuw-West', 
    'Osdorpplein e.o.': 'Stadsdeel Nieuw-West',
    'Calandlaan/Lelylaan': 'Stadsdeel Nieuw-West', 
    'Lucas/Andreasziekenhuis e.o.': 'Stadsdeel Nieuw-West',
    'Dorp Sloten': 'Stadsdeel Nieuw-West', 
    'Belgiëplein e.o.': 'Stadsdeel Nieuw-West',
    'BurgemeesterTellegenbuurt-West': 'Stadsdeel Zuid',
    'Veluwebuurt': 'Stadsdeel Zuid',
    'Buitenveldert-Zuid-Midden': 'Stadsdeel Zuid', 
    'Cruquiusbuurt': 'Stadsdeel Oost',
    'Weesperzijde-Midden/Zuid' : 'Stadsdeel Oost', 
    'Weesperzijde-Noord': 'Stadsdeel Oost',
    'Timorpleinbuurt-Noord': 'Stadsdeel Oost', 
    'Ambonpleinbuurt': 'Stadsdeel Oost', 
    'Timorpleinbuurt-Zuid': 'Stadsdeel Oost',
    'Makassarpleinbuurt': 'Stadsdeel Oost', 
    'Sumatraplantsoenbuurt': 'Stadsdeel Oost', 
    'Sportheldenbuurt': 'Stadsdeel Oost',
    'Oostpunt Zeeburgereiland': 'Stadsdeel Oost', 
    'Sluisbuurt': 'Stadsdeel Oost', 
    'Baaibuurt-West': 'Stadsdeel Oost',
    'Bovendiep/Diemerpark': 'Stadsdeel Oost', 
    'Joris Ivenspleinbuurt': 'Stadsdeel Oost',
    'Ed Pelsterparkbuurt': 'Stadsdeel Oost', 
    'Rieteilanden-West': 'Stadsdeel Oost', 
    'Havenkwartier IJburg': 'Stadsdeel Oost',
    'Theo van Goghparkbuurt': 'Stadsdeel Oost', 
    'Circus/Kermisbuurt': 'Stadsdeel Noord',
    'Buiksloterham-Noord': 'Stadsdeel Noord', 
    'Buiksloterham-Zuid': 'Stadsdeel Noord', 
    'Klaprozenbuurt': 'Stadsdeel Noord',
    'Markengouw-Noord': 'Stadsdeel Noord', 
    'G-buurt-Oost': 'Stadsdeel Zuidoost', 
    'Grunder/Koningshoef' : 'Stadsdeel Zuidoost',
    'Amstel III deel A/B-Noord': 'Stadsdeel Zuidoost',
    'Amstel III deel A/B-Zuid': 'Stadsdeel Zuidoost',
    'Afrikahaven' : 'Westpoort', 
    'Coenhaven' : 'Westpoort', 
    'Vlothaven': 'Westpoort', 
    'Minervahaven-Noordoost': 'Westpoort',
    'Minervahaven-Noordwest': 'Westpoort', 
    'Hemknoop-Noord': 'Westpoort', 
    'Isolatorbuurt': 'Stadsdeel West',
    'Contactbuurt': 'Stadsdeel West', 
    'Hemknoop-Zuid': 'Stadsdeel West', 
    'Zaanstraat emplacement': 'Stadsdeel West',
    'Sloterdijk De Heining': 'Stadsdeel Nieuw-West', 
    'Sloterdijk Poort-Noord': 'Stadsdeel Nieuw-West',
    'Sloterdijk Poort-Zuid': 'Stadsdeel Nieuw-West', 
    'Sloterdijk Rhônepark': 'Stadsdeel Nieuw-West',
    'Sloterdijk Donauhoek': 'Stadsdeel Nieuw-West', 
    'Meerwaldtbuurt': 'Stadsdeel Nieuw-West', 
    'Baaibuurt-Oost': 'Stadsdeel Oost',
    'Melkweg/Oostzanerwerf': 'Stadsdeel Noord',
    'Cornelis Douwesterrein-West': 'Stadsdeel Noord',
    'Cornelis Douwesterrein-Oost': 'Stadsdeel Noord', 
    'Werengouw-Noord': 'Stadsdeel Noord',
    'Hamerstraatkwartier-West': 'Stadsdeel Noord', 
    'Hamerstraatkwartier-Oost': 'Stadsdeel Noord',
    'Weespersluis-Noord': 'Weesp', 
    'Waterrijk': 'Weesp', 
    'Lanenrijk': 'Weesp', 
    'Vechtrijk': 'Weesp',
    'Leeuwenveld': 'Weesp', 
    'Bedrijventerrein Nijverheidslaan': 'Weesp',
    'Bedrijventerrein-Noord Weesp': 'Weesp', 
    'Hogewey-Noord': 'Weesp', 
    'Hogewey-Midden': 'Weesp',
    'Hogewey-Zuid': 'Weesp', 
    'Weesp Dichtersbuurt': 'Weesp', 
    'Weesp Schildersbuurt': 'Weesp',
    'Herensingelkwartier': 'Weesp', 
    'Bedrijventerrein Van Houten': 'Weesp',
    'Weesp Centrum': 'Weesp', 
    'Weesp-Zuid I': 'Weesp', 
    'Weesp-Zuid II': 'Weesp', 
    'De Klomp': 'Weesp',
    'Aetsveldsepolder': 'Weesp', 
    'Aetsveld-Zuid': 'Weesp', 
    'Aetsveld-Noord': 'Weesp',
    'Oostelijke Vechtoever': 'Weesp', 
    'Amstel III deel C/D-Noord': 'Stadsdeel Zuidoost',
    'Amstel III deel C/D-Zuid': 'Stadsdeel Zuidoost', 
    'G-buurt-Noord': 'Stadsdeel Zuidoost', 
    'K-buurt-Zuidoost' : 'Stadsdeel Zuidoost'
}

In [18]:
df_join_3 = pd.concat([df_join_1.dropna(), df_join_2])

In [19]:
df_join_3.loc[df_join_3['stadsdeel_onderverdeling'].isna(), 'stadsdeel_onderverdeling'] = df_join_3.loc[df_join_3['stadsdeel_onderverdeling'].isna(), 'buurt'].map(buurt_unmatched_to_subdivision)

In [20]:
df_join_3.loc[df_join_3['stadsdeel'].isna(), 'stadsdeel'] = df_join_3.loc[df_join_3['stadsdeel'].isna(), 'buurt'].map(buurt_unmatched_to_stadsdeel)

In [21]:
df_join_3.isna().sum()

buurt                       0
wijk                        0
stadsdeel_onderverdeling    0
stadsdeel                   0
dtype: int64

In [22]:
df_join_3['stadsdeel_onderverdeling'] = df_join_3['stadsdeel_onderverdeling'].replace('Driemond', 'Weesp')
df_join_3['stadsdeel_onderverdeling'] = df_join_3['stadsdeel_onderverdeling'].replace('Westelijk Havengebied', 'Westpoort')

In [25]:
df_join_3 = df_join_3.drop(columns=['wijk'])

In [26]:
df_join_3.to_csv('geodata/buurt_stadsdeel_mapping.csv', sep=';', index=False)

In [24]:
df_join_3.value_counts('stadsdeel_onderverdeling')

stadsdeel_onderverdeling
Centrum-Oost                              35
Centrum-West                              34
Oud Zuid                                  34
Oud West / De Baarsjes                    30
Watergraafsmeer                           27
Geuzenveld-Slotermeer-Sloterdijken        27
Oost                                      25
Westerpark                                25
De Pijp / Rivierenbuurt                   25
Oud-Noord                                 25
Weesp                                     24
West                                      19
Slotervaart                               19
Bijlmer Centrum                           18
Indische Buurt / Oostelijk Havengebied    18
Buitenveldert / Zuidas                    17
IJburg / Eiland Zeeburg                   15
Bos en Lommer                             15
Osdorp                                    14
Bijlmer Oost                              14
Gaasperdam                                14
Westpoort                     